In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import random

In [2]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
##GPU 사용 시
device = torch.device("cuda:0")

In [5]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [6]:
#!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
#!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [7]:
dataset_train = nlp.data.TSVDataset("train.tsv", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("val.tsv", field_indices=[1,2], num_discard_samples=1)

In [8]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [9]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [transform([i[label_idx]]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + self.labels[i])

    def __len__(self):
        return (len(self.labels))

In [23]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 300
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [24]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [25]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [26]:
class BERTSeq2Seq(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTSeq2Seq, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        
        self.embed_size = 128                 
        self.embedding = nn.Embedding(len(vocab), self.embed_size)
        #self.classifier = nn.Linear(hidden_size , num_classes)
        self.decoder = nn.GRU(self.embed_size, hidden_size)
        self.out = nn.Linear(hidden_size, len(vocab))
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids, output_ids, output_valid_lengths, teacher_forcing_ratio=0.5):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
        if self.dr_rate:
            hidden = self.dropout(pooler)
            
        hidden = hidden.unsqueeze(0)                        
            
        output_id = output_ids[0,:]        
        trg_len = output_ids.shape[0]
        batch_size = output_ids.shape[1]
        trg_vocab_size = len(vocab)
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            
            output_id_embedded = self.embedding(output_id.unsqueeze(0))            
            
            output, hidden = self.decoder(output_id_embedded, hidden)
            
            pred = self.out(output.squeeze(0))
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = pred
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = pred.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            output_id = output_ids[t] if teacher_force else top1
        
        return outputs

In [27]:
model = BERTSeq2Seq(bertmodel,  dr_rate=0.5).to(device)

In [28]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [29]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss(ignore_index=vocab['[PAD]'])

In [30]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [31]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [32]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, output_ids, output_valid_lengths, _) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
                       
        label_ids = output_ids.transpose(1, 0).long().to(device)
        
        output_valid_lengths = output_valid_lengths
        
        out = model(token_ids, valid_length, segment_ids, label_ids, output_valid_lengths)
        
        out = out.view(-1, out.shape[2])
        label_ids = label_ids.flatten()
                
        loss = loss_fn(out, label_ids)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label_ids)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, output_ids, output_valid_lengths, _) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
                       
        label_ids = output_ids.transpose(1, 0).long().to(device)
        
        output_valid_lengths = output_valid_lengths
        
        out = model(token_ids, valid_length, segment_ids, label_ids, output_valid_lengths)
        
        out = out.view(-1, out.shape[2])
        label_ids = label_ids.flatten()
        
        test_acc += calc_accuracy(out, label_ids)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 8.993640899658203 train acc 0.0

epoch 1 train acc 4.438920454545455e-05



epoch 1 test acc 1.4361213235294117e-05


epoch 2 batch id 1 loss 8.95656681060791 train acc 0.0

epoch 2 train acc 0.0036177201704545455



epoch 2 test acc 0.018817976409313725


epoch 3 batch id 1 loss 8.857796669006348 train acc 0.017822265625

epoch 3 train acc 0.06082134824810606


...

epoch 8 batch id 1 loss 4.209588527679443 train acc 0.06103515625

epoch 8 train acc 0.08410496567234849



epoch 8 test acc 0.0849465762867647


epoch 9 batch id 1 loss 4.132456302642822 train acc 0.061279296875

epoch 9 train acc 0.08471605705492424



epoch 9 test acc 0.0868231081495098


epoch 10 batch id 1 loss 4.071106910705566 train acc 0.061767578125
